<a href="https://colab.research.google.com/github/davidenko2000/ProjectR/blob/main/Resnet18_mnist512.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1 #sto znaci ovaj expansion?

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        #dimenzija jezgre odnosno matrice koja se pomice po ulaznoj i stvara mapu znacajki, 
        #padding nadopunjuje rubove, bias je false jer se koristi BatchNorm, stride je broj koraka(redaka/stupaca) koliko se pomice jezgra

        self.bn1 = nn.BatchNorm2d(planes)#normalizacija pomice vrijednosti u ovisnosti o srednjoj vrij.
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()#kombinira module
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )
        #u ovaj if se ulazi kod svakog osim prvo bloka
        #TODO nadopuniti opis, sto znaci self.expansion? 

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out
        # CONV1 -> BN1 -> ReLu -> CONV2 -> BN2 = F(X)
        # F(x) + shorcut -> ReLu
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):#koliko klasa imamo na kraju
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(1, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)#zbog grayscale inpanes je 1, za cifar 3
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)# flattening

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)#listu od dva elementa, prvi i drugi element su strideovi 
        layers = []
        for stride in strides:#svi u layeru imaju stride 1, osim prvog koji ima 2
            layers.append(block(self.in_planes, planes, stride))#appenda na listu blok
            self.in_planes = planes * block.expansion#pridruzivanje planesa in_planes, mnoezenjem s 1?
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)#out je jezgra, a 4 je stride, odnosno korak
        out = out.view(out.size(0), -1)#reshape tensora prije nego ide dalje, -1 znaci da ne znamo broj redaka/stupaca
        out = self.linear(out)#flattening prije fully connected layera
        return out
        # CONV1 -> BN1 -> Layer1(sa dva bloka) -> Layer2(sa dva bloka) -> Layer3(sa dva bloka) -> Layer4(sa dva bloka)
        # AVGPOOL -> reshape -> flattening (linear) ili downsample

def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])#u svakom sloju koliko je blokova

In [10]:
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time

best_acc = 0  
start_epoch = 0  

# Data
print('==> Preparing data..')
transform_train = transforms.Compose([#spaja transformacije zajedno
    #transforms.RandomCrop(32, padding=4),#slučajno cropa dijelove slike
    transforms.RandomCrop(28, padding=4),#za mnist
    transforms.RandomHorizontalFlip(),#ili flipa ili ne
    transforms.ToTensor(),
    #transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),#prvi tupple su meanovi, 
    #a drugi stand devijacije, ovo su za cifar10, ima 3 vrijednosti (visina, sirina, boja), za mnist su dvije
    transforms.Normalize((0.1307,), (0.3081,)),# ovo je za mnist
])


#trainset = torchvision.datasets.CIFAR10(
 #   root='./data', train=True, download=True, transform=transform_train)#skinut cifar i mnist na google drive

trainset = torchvision.datasets.MNIST(
    root='./data', train=True, download=True, transform=transform_train)

trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=512, shuffle=True, num_workers=2)
#hiperparametri - epohe i batchsize
#classes = ('plane', 'car', 'bird', 'cat', 'deer',
 #         'dog', 'frog', 'horse', 'ship', 'truck')

classes = ('0', '1', '2', '3', '4',
           '5', '6', '7', '8', '9')

#Model
print('==> Building model..')

net = ResNet18()
net = net.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01,
                      momentum=0.9, weight_decay=5e-4)#prouciti momentum

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)#za smanjivanje learning ratea, zasto cosine

start_time = time.time()
# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    train_loss = 0
    correct = 0
    total = 0
    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))   
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.cuda(), targets.cuda()
        optimizer.zero_grad()#postavlja sve vrijednosti na pocetku na 0, da ne kompromitira
        outputs = net(inputs)
        loss = criterion(outputs, targets)#računa gubitak uz pomoc negativne log izglednosti
        loss.backward()#propagiramo nazad u mrezi
        optimizer.step()#natjeramo da iterira po svim parametrira tensora

        train_loss += loss.item()#zbraja gubitak
        _, predicted = outputs.max(1)#odabiremo neuron s najvecom aktivacijom
        total += targets.size(0)#racunamo kolko je tre
        correct += predicted.eq(targets).sum().item()#usporeduje s targetima i zbraja koliko je tocnih

        print(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                     % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))#redni broj batcha, velicina cijelog dataset, prosjecan gubitak, tocnost,tocno, ukupno 
     

for epoch in range(start_epoch, start_epoch+10):
      train(epoch)
      scheduler.step()

==> Preparing data..
==> Building model..

Epoch: 0
Time elapsed: 0.00 min
0 118 Loss: 2.353 | Acc: 11.133% (57/512)
1 118 Loss: 2.314 | Acc: 12.793% (131/1024)
2 118 Loss: 2.284 | Acc: 15.820% (243/1536)
3 118 Loss: 2.253 | Acc: 17.432% (357/2048)
4 118 Loss: 2.222 | Acc: 18.555% (475/2560)
5 118 Loss: 2.206 | Acc: 19.108% (587/3072)
6 118 Loss: 2.168 | Acc: 20.424% (732/3584)
7 118 Loss: 2.139 | Acc: 21.606% (885/4096)
8 118 Loss: 2.105 | Acc: 22.982% (1059/4608)
9 118 Loss: 2.076 | Acc: 23.906% (1224/5120)
10 118 Loss: 2.049 | Acc: 25.444% (1433/5632)
11 118 Loss: 2.027 | Acc: 26.497% (1628/6144)
12 118 Loss: 2.001 | Acc: 28.050% (1867/6656)
13 118 Loss: 1.973 | Acc: 29.771% (2134/7168)
14 118 Loss: 1.943 | Acc: 31.289% (2403/7680)
15 118 Loss: 1.913 | Acc: 32.849% (2691/8192)
16 118 Loss: 1.881 | Acc: 34.639% (3015/8704)
17 118 Loss: 1.847 | Acc: 36.382% (3353/9216)
18 118 Loss: 1.811 | Acc: 38.106% (3707/9728)
19 118 Loss: 1.775 | Acc: 39.482% (4043/10240)
20 118 Loss: 1.737 | Acc